<a href="https://colab.research.google.com/github/hrk022/FakeNews-Detection/blob/main/FakeNews.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Dense, Flatten
from sklearn.model_selection import train_test_split


In [ ]:
df_true = pd.read_csv("/content/True.csv")
df_fake = pd.read_csv("/content/Fake.csv")

In [ ]:
df_true.head()

,title,text,subject,date
0,"As U.S. budget fight looms, Republicans flip t...",WASHINGTON (Reuters) - The head of a conservat...,politicsNews,"December 31, 2017"
1,U.S. military to accept transgender recruits o...,WASHINGTON (Reuters) - Transgender people will...,politicsNews,"December 29, 2017"
2,Senior U.S. Republican senator: 'Let Mr. Muell...,WASHINGTON (Reuters) - The special counsel inv...,politicsNews,"December 31, 2017"
3,FBI Russia probe helped by Australian diplomat...,WASHINGTON (Reuters) - Trump campaign adviser ...,politicsNews,"December 30, 2017"
4,Trump wants Postal Service to charge 'much mor...,SEATTLE/WASHINGTON (Reuters) - President Donal...,politicsNews,"December 29, 2017"


In [ ]:
df_true.shape

(21417, 4)

In [ ]:
df_true["text"][0]

'WASHINGTON (Reuters) - The head of a conservative Republican faction in the U.S. Congress, who voted this month for a huge expansion of the national debt to pay for tax cuts, called himself a “fiscal conservative” on Sunday and urged budget restraint in 2018. In keeping with a sharp pivot under way among Republicans, U.S. Representative Mark Meadows, speaking on CBS’ “Face the Nation,” drew a hard line on federal spending, which lawmakers are bracing to do battle over in January. When they return from the holidays on Wednesday, lawmakers will begin trying to pass a federal budget in a fight likely to be linked to other issues, such as immigration policy, even as the November congressional election campaigns approach in which Republicans will seek to keep control of Congress. President Donald Trump and his Republicans want a big budget increase in military spending, while Democrats also want proportional increases for non-defense “discretionary” spending on programs that support educat

In [ ]:
df_true['label'] = 1
df_fake['label'] = 0

df = pd.concat([df_true, df_fake], axis=0).reset_index(drop=True)

In [ ]:
X = df['text']
y = df['label']

In [ ]:
print(pd.Series(y).value_counts())

label
0    23481
1    21417
Name: count, dtype: int64


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
tokenizer = Tokenizer(num_words=5000)
tokenizer.fit_on_texts(X_train)

In [ ]:
X_train_sequence = tokenizer.texts_to_sequences(X_train)
X_test_sequence = tokenizer.texts_to_sequences(X_test)

In [ ]:
maxlen = 300
X_train_padded = pad_sequences(X_train_sequence,padding="post",maxlen=maxlen)
X_test_padded = pad_sequences(X_test_sequence,padding="post",maxlen=maxlen)

In [ ]:
from tensorflow.keras.layers import LSTM, Dropout

model = Sequential()
model.add(Embedding(input_dim=5000, output_dim=64, input_length=maxlen))
model.add(LSTM(64, return_sequences=True))
model.add(LSTM(32))
model.add(Dense(32, activation="relu"))
model.add(Dropout(0.3))
model.add(Dense(1, activation="sigmoid"))

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


In [ ]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:
history = model.fit(X_train_padded, y_train, epochs=5, batch_size=32, validation_data=(X_test_padded, y_test))

Epoch 1/5
1123/1123 ━━━━━━━━━━━━━━━━━━━━ 388s 340ms/step - accuracy: 0.7994 - loss: 0.4146 - val_accuracy: 0.9545 - val_loss: 0.1524
Epoch 2/5
1123/1123 ━━━━━━━━━━━━━━━━━━━━ 367s 326ms/step - accuracy: 0.9604 - loss: 0.1286 - val_accuracy: 0.9811 - val_loss: 0.0596
Epoch 3/5
1123/1123 ━━━━━━━━━━━━━━━━━━━━ 397s 339ms/step - accuracy: 0.9950 - loss: 0.0179 - val_accuracy: 0.9938 - val_loss: 0.0180
Epoch 4/5
1123/1123 ━━━━━━━━━━━━━━━━━━━━ 380s 338ms/step - accuracy: 0.9963 - loss: 0.0101 - val_accuracy: 0.9914 - val_loss: 0.0273
Epoch 5/5
1123/1123 ━━━━━━━━━━━━━━━━━━━━ 379s 335ms/step - accuracy: 0.9985 - loss: 0.0051 - val_accuracy: 0.9860 - val_loss: 0.0587


In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.sequence import pad_sequences

def prediction(text):
    # Convert input text to sequence
    text_seq = tokenizer.texts_to_sequences([text])

    # Pad the sequence (ensure maxlen matches training data)
    text_padded = pad_sequences(text_seq, maxlen=300, padding="post")

    # Make prediction
    prediction = model.predict(text_padded)[0][0]  # Extract scalar value

    # Return result
    return "Real News" if prediction > 0.5 else "Fake News"

# Get user input and predict
text = input("Enter text: ")
result = prediction(text)
print(f"Prediction: {result}")


In [ ]:
import matplotlib.pyplot as plt
plt.plot(history.history["loss"])
plt.plot(history.history["val_loss"])
plt.title("Model Loss")
plt.ylabel("Loss")
plt.xlabel("Epoch")
plt.legend(["Train", "Validation"], loc="upper right")
plt.show()